In [1]:
import os
os.environ['PROJECT_ID'] = 'able-analyst-416817'
print(os.environ.get('PROJECT_ID'))

able-analyst-416817


In [2]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
TAG_NAME = 'masterv6'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-fine-tunning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
BUCKET_NAME = 'able-analyst-416817-chatbot-v1'
# BUCKET_URI = f"gs://{BUCKET_NAME}"
FINE_TUNE_FLAG=  False
EPOCHS=12
MODEL_NAME="gemma_2b_en"


In [ ]:
user = os.getenv("USER_NAME", "")
print(os.path.join(user , "jhdbcjd"))
print(os.path.join("cdsac", user))


In [ ]:
CONTAINER_IMAGE_RUNNING_APP

In [ ]:
substitutions="""
_CONTAINER_IMAGE_NAME_DATA_PREP={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_NAME_DATA_PREP,
           TAG_NAME, 
           ).strip()
print(substitutions)

# Runs the data_preparation component image. (Development, when tested should be moved to the main cloudbuild in the project folder)
# Pay attention to the "." after summit. Might need some changes when move to the master pipeline.
!gcloud builds submit . --timeout=15m --config "components/data_preparation/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}
# DO not forget the tag

In [ ]:
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-data-preparation:latest data_ingestion.py --bucket-name 'able-analyst-416817-chatbot-v1' --directory 'input_data/andrehpereh'

In [ ]:
import vertexai
print(vertexai.__version__)

In [ ]:
substitutions="""
_CONTAINER_IMAGE_NAME_FINE_TUNE={},\
_KAGGLE_USERNAME={},\
_KAGGLE_KEY={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_NAME_FINE_TUNE,
           KAGGLE_USERNAME,
           KAGGLE_KEY,
           TAG_NAME, 
           ).strip()
print(substitutions)

# Builds image
!gcloud builds submit . --config "components/fine_tunning/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
import json

data = ["Sender: FoooodddAndres Perez: Coming :)", "Sender: Can I maybe borrow your iron? Andres Perez: It\'s not my iron But yeah haha Or is it?"]
model_paths = """{"finetuned_model_dir": "./gemma_2b_en", "finetuned_weights_path": "./gemma_2b_en/model.weights.h5"}"""
print(len(data))
model_paths = json.dumps(model_paths)
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest trainer.py --data {data} --model-paths {model_paths}
#!python ./components/fine_tunning/trainer.py --data {data} --model-paths {model_paths}
#!python ./components/fine_tunning/conversion_function.py --weights-file {model_paths_and_config['finetuned_weights_path']} --size {model_paths_and_config['model_size']} --vocab-path {model_paths_and_config['finetuned_vocab_path']} --output-dir {model_paths_and_config['huggingface_model_dir']}

In [3]:
substitutions="""
_CONTAINER_IMAGE_RUNNING_APP={},\
TAG_NAME={}\
""".format(
           CONTAINER_IMAGE_RUNNING_APP,
           TAG_NAME, 
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/app_flask/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

_CONTAINER_IMAGE_RUNNING_APP=gemma-chatbot-running-app,TAG_NAME=masterv6
Creating temporary tarball archive of 58 file(s) totalling 673.4 KiB before compression.
Uploading tarball of [.] to [gs://able-analyst-416817_cloudbuild/source/1712079603.156165-a3b59de8a4594249a3877314948ec089.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/able-analyst-416817/locations/us-central1/builds/ae11ae33-f738-42f0-86ff-7725330929f5].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/ae11ae33-f738-42f0-86ff-7725330929f5?project=24796876098 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ae11ae33-f738-42f0-86ff-7725330929f5"

FETCHSOURCE
Fetching storage object: gs://able-analyst-416817_cloudbuild/source/1712079603.156165-a3b59de8a4594249a3877314948ec089.tgz#1712079603804234
Copying gs://able-analyst-416817_cloudbuild/source/1712079603.156165-a3b59de8a4594249a3877314948ec089.tgz#1712079603804234..

In [ ]:
!kfp dsl compile --py './components/pipeline/pipeline/pipeline.py' --output "fine_tune_pipeline_test.json" --function fine_tune_pipeline

In [ ]:
_CONTAINER_IMAGE_RUNNING_APP

In [ ]:
!pwd

In [ ]:
substitutions="""
_CONTAINER_IMAGE_NAME_PIPELINE={},\
TAG_NAME={},\
_BUCKET_NAME={},\
_FINE_TUNE_FLAG={},\
_EPOCHS={},\
_MODEL_NAME={}
""".format(
           CONTAINER_IMAGE_NAME_PIPELINE,
           TAG_NAME,
           BUCKET_NAME,
           FINE_TUNE_FLAG,
           EPOCHS,
           MODEL_NAME
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/pipeline/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
# !python "./components/pipeline/pipeline.py"

In [ ]:
substitutions=f"_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},TAG_NAME={TAG_NAME}"
print(substitutions)
!gcloud builds submit . --config "./components/app_flask/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
PROD_TAG = 'master_v4'
substitutions=f"_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},_PROD_TAG={PROD_TAG}"
!gcloud builds submit . --config "cloudbuil_cloudrun.yaml" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
from cluodbuild_compiler import merge_cloudbuild_files, find_missing_elements
child_files = [ 
    "./components/data_preparation/cloudbuild.yaml", 
    "./components/fine_tunning/cloudbuild.yaml", 
    "./components/pipeline/cloudbuild.yaml",
    "./components/app_flask/cloudbuild.yaml"
]

descriptions = [ 
    "Data Preparation",
    "Model Training", 
    "Vertex AI Pipeline Deployment",
    "App flask application (Cloud Run)"
]
all_variables = merge_cloudbuild_files(child_files, descriptions, timeout_hours = 20)

In [ ]:
substitutions=f"_CONTAINER_IMAGE_NAME_DATA_PREP={CONTAINER_IMAGE_NAME_DATA_PREP},_CONTAINER_IMAGE_NAME_FINE_TUNE={CONTAINER_IMAGE_NAME_FINE_TUNE}\
,_CONTAINER_IMAGE_NAME_PIPELINE={CONTAINER_IMAGE_NAME_PIPELINE},_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP},_BUCKET_NAME={BUCKET_NAME}\
_KAGGLE_USERNAME={KAGGLE_USERNAME},_KAGGLE_KEY={KAGGLE_KEY},_KAGGLE_USERNAME={KAGGLE_USERNAME},_FINE_TUNE_FLAG={FINE_TUNE_FLAG},_EPOCHS={EPOCHS},_MODEL_NAME={MODEL_NAME},TAG_NAME={TAG_NAME}"
print(substitutions)

In [ ]:
missing_elements = find_missing_elements(all_variables, substitutions)
print(missing_elements)


In [ ]:
!gcloud builds submit . --config "./master_cloudbuild.json" --substitutions {substitutions} --region={GCP_REGION}

In [ ]:
from werkzeug.datastructures import FileStorage

In [ ]:
from google.cloud import bigquery, storage, pubsub_v1

In [ ]:
!gcloud pubsub subscriptions pull

In [ ]:
!gcloud pubsub subscriptions pull --auto-ack your-pipeline-trigger-topic-sub

In [ ]:
import os
from google.cloud import pubsub_v1

# Project and subscription details
project_id = "able-analyst-416817"  
subscription_name = "your-pipeline-trigger-topic-sub"

# Create Subscriber client
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_name)

def callback(message):
    print(f"Received message: {message.data}")
    message.ack()  # Acknowledge the message

# Pulling messages
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

# Wrap subscriber in a 'with' block to automatically close the client when done.
with subscriber:
    try:
        # When `timeout` is notset, result() will block indefinitely,
        # unless an exception is encountered.
        streaming_pull_future.result()  # Continuously pulls messages
    except TimeoutError: 
        streaming_pull_future.cancel()  # Trigger shutdown
        streaming_pull_future.result()  # Block until shutdown is complete 


In [ ]:
! gcloud functions logs read \
  --gen2 \
  --region='us-central1' \
  --limit=5 \
  'function-1'

In [ ]:
!gcloud builds submit . --config "./master_cloudbuild.json" --substitutions {substitutions} --region={GCP_REGION}